In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import matplotlib.pyplot as plt
from sklearn.metrics import r2_score

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [3]:
dataset = pd.read_csv('/home/urwa/Documents/side_projects/urban/data/featureData/com_penn.csv')

In [4]:
dataset.shape

(8757, 113)

In [5]:
dataset.head(3)

,Date,Hour,0.0,0.1,0.2,1.0,1.1,1.2,1.3,2.0,...,4.1_lag_3,4.2_lag_3,4.3_lag_3,4.4_lag_3,4.5_lag_3,5.0_lag_3,5.1_lag_3,5.2_lag_3,5.3_lag_3,arrival_lag_3
0,2018-01-01,3,134,37,126,15,16,8,35,3,...,2.0,4.0,2.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0
1,2018-01-01,4,82,22,71,6,11,3,21,3,...,4.0,2.0,7.0,0.0,12.0,0.0,0.0,1.0,0.0,1.0
2,2018-01-01,5,53,11,24,8,6,1,13,3,...,4.0,1.0,3.0,0.0,32.0,0.0,0.0,0.0,0.0,1.0


In [6]:
lag_columns = [c for c in dataset.columns if 'lag' in c]
len(lag_columns)

75

In [7]:
dataset = dataset[[c for c in dataset.columns if c not in lag_columns]]
dataset.shape

(8757, 38)

In [8]:
DateColumns = ['Date']

ext_columns = ['Dow', 'arrival','maxtemp', 'mintemp', 'avgtemp', 'departure', 'hdd',
       'cdd', 'participation', 'newsnow', 'snowdepth', 'ifSnow']

targetColumns = [c for c in dataset.columns if c not in ext_columns and \
                c not in DateColumns and c not in lag_columns and c != 'Hour']
len(targetColumns)

24

In [9]:
features_cols = [c for c in dataset.columns if c not in targetColumns and c not in DateColumns]
len(features_cols)

13

In [10]:
sep = int(0.75*len(dataset))
print(sep)


trainData = dataset[:sep]
testData = dataset[sep:]

print(trainData.shape)
print(testData.shape)

6567
(6567, 38)
(2190, 38)


In [11]:
X_train = trainData[features_cols].values
X_train = torch.tensor(X_train).float().to(device)
print(X_train.shape)

y_train = trainData[targetColumns].values
y_train = torch.tensor(y_train).float().to(device)
print(y_train.shape)

X_test = testData[features_cols].values
X_test = torch.tensor(X_test).float().to(device)
print(X_test.shape)

y_test = testData[targetColumns].values
y_test = torch.tensor(y_test).float().to(device)
print(y_test.shape)

torch.Size([6567, 13])
torch.Size([6567, 24])
torch.Size([2190, 13])
torch.Size([2190, 24])


In [12]:
def create_inout_sequences(x,y, tw):
    inout_seq = []
    L = len(x)
    for i in range(L-tw):
        train_seq_x = x[i:i+tw]
        train_seq_y = y[i:i+tw]
        train_seq = torch.cat((train_seq_x,train_seq_y),axis=1)
        
#         train_label = y[i+tw:i+tw+1]
        train_label = y[i+1:i+tw+1]
        inout_seq.append((train_seq ,train_label))
    return inout_seq

In [13]:
bptt = 6

In [14]:
train_inout_seq = create_inout_sequences(X_train,y_train, bptt)

In [15]:
train_inout_seq[0][0].shape,train_inout_seq[0][1].shape

(torch.Size([6, 37]), torch.Size([6, 24]))

In [16]:
class LSTM(nn.Module):
    def __init__(self, input_size=1, hidden_layer_size=100, output_size=1, layers=1):
        super().__init__()
        self.hidden_layer_size = hidden_layer_size

        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_layer_size, num_layers=layers)

        self.linear = nn.Linear(hidden_layer_size, output_size)

        self.hidden_cell = (torch.zeros(layers,1,self.hidden_layer_size),
                            torch.zeros(layers,1,self.hidden_layer_size))

    def forward(self, input_seq):
        lstm_out, self.hidden_cell = self.lstm(input_seq.view(len(input_seq) ,1, -1), self.hidden_cell)
        predictions = self.linear(lstm_out.view(len(input_seq), -1))
#         return predictions[-1]
        return predictions

In [17]:
layers = 2
model = LSTM(input_size = 37, hidden_layer_size=258, output_size=24, layers=layers).to(device)
loss_function = nn.L1Loss()   
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [18]:
epochs = 100


for i in range(epochs):
    for seq, labels in train_inout_seq:
        optimizer.zero_grad()
        model.hidden_cell = (torch.zeros(layers, 1, model.hidden_layer_size).to(device),
                        torch.zeros(layers, 1, model.hidden_layer_size).to(device))

        y_pred = model(seq)

        single_loss = loss_function(y_pred, labels)
        single_loss.backward()
        optimizer.step()

    if i%25 == 1:
        print(f'epoch: {i:3} loss: {single_loss.item():10.8f}')

print(f'epoch: {i:3} loss: {single_loss.item():10.10f}')

epoch:   1 loss: 4.41970015
epoch:  26 loss: 3.61195540
epoch:  51 loss: 3.78327012
epoch:  76 loss: 3.48215795
epoch:  99 loss: 3.1055917740


In [19]:
test_inout_seq = create_inout_sequences(X_test,y_test, bptt)

In [20]:
model.eval()
prediction = []
with torch.no_grad():
    for seq, labels in test_inout_seq:
        model.hidden = (torch.zeros(layers, 1, model.hidden_layer_size),
                        torch.zeros(layers, 1, model.hidden_layer_size))
        prediction.append(model(seq)[-1])

In [21]:
y_test_ = torch.stack([labels[-1] for seq, labels in test_inout_seq], axis=0).detach().cpu().numpy()
y_pred_ = torch.stack(prediction).detach().cpu().numpy()

In [22]:
y_test_.shape, y_pred_.shape

((2184, 24), (2184, 24))

In [23]:
r2_score(y_test_, y_pred_, multioutput='variance_weighted')

0.8318228415696451

### Edge Prediction

In [24]:
edge_prediction_df = pd.DataFrame(y_pred_)
edge_prediction_df.columns = testData[6:][targetColumns].columns
edge_prediction_df.head(2)

,0.0,0.1,0.2,1.0,1.1,1.2,1.3,2.0,2.1,2.2,...,4.0,4.1,4.2,4.3,4.4,4.5,5.0,5.1,5.2,5.3
0,95.815331,30.891098,69.847862,8.350304,13.731751,4.872939,17.321756,4.764876,3.677532,13.611024,...,0.047884,4.099535,3.527463,5.614175,0.051364,15.547918,0.005454,-0.033349,0.008163,0.009803
1,43.356762,13.556379,26.400442,4.179013,5.831906,2.335419,9.302417,2.917875,1.665354,6.798444,...,0.004019,2.296136,2.316316,2.602067,0.005382,7.876994,0.001669,-0.016429,-0.005528,-0.031352


In [25]:
edge_prediction_df.columns

Index(['0.0', '0.1', '0.2', '1.0', '1.1', '1.2', '1.3', '2.0', '2.1', '2.2',
       '2.3', '3.0', '3.1', '3.2', '4.0', '4.1', '4.2', '4.3', '4.4', '4.5',
       '5.0', '5.1', '5.2', '5.3'],
      dtype='object')

In [26]:
zone_weights = pd.read_csv('/home/urwa/Documents/side_projects/urban/data/featureData/com_penn_weights.csv')

In [27]:
zone_weights['Borough'] = zone_weights['Borough'].astype(str)

In [28]:
boroughs = list(edge_prediction_df.columns)
for bor in boroughs:
    print(bor)
    
    weight_df = zone_weights[zone_weights.Borough == bor]
    
    print(len(weight_df.DOLocationID))
    
    for b_zone,z_weight in zip(weight_df.DOLocationID.values,weight_df.zone_weight.values):        
        edge_prediction_df[b_zone] = edge_prediction_df[bor] * z_weight

0.0
14
0.1
4
0.2
21
1.0
15
1.1
17
1.2
20
1.3
10
2.0
14
2.1
19
2.2
8
2.3
10
3.0
6
3.1
4
3.2
8
4.0
1
4.1
18
4.2
23
4.3
12
4.4
5
4.5
9
5.0
4
5.1
8
5.2
4
5.3
3


In [29]:
select_cols = [c for c in edge_prediction_df.columns if c not in boroughs]
edge_prediction_df = edge_prediction_df[select_cols]
edge_prediction_df.shape

(2184, 257)

In [30]:
edge_testData = pd.read_csv('/home/urwa/Documents/side_projects/urban/data/featureData/penn.csv')
edge_testData = edge_testData[sep+bptt:]

lag_columns = [c for c in edge_testData.columns if 'lag' in c]
edge_testData = edge_testData[[c for c in edge_testData.columns if c not in lag_columns]]

edgeColumns = [c for c in edge_testData.columns if c not in ext_columns and \
                c not in DateColumns and c not in lag_columns and c != 'Hour']

edge_testData = edge_testData[edgeColumns]

edge_testData.shape

(2184, 257)

In [31]:
edge_prediction_df.columns

Index([  1,  48,  50, 100, 137, 138, 161, 162, 163, 164,
       ...
       118, 176, 204,  23, 156, 187, 251,   6, 172, 214],
      dtype='object', length=257)

In [32]:
edge_testData.columns

Index(['1', '10', '100', '101', '102', '106', '107', '108', '109', '11',
       ...
       '90', '91', '92', '93', '94', '95', '96', '97', '98', '99'],
      dtype='object', length=257)

In [34]:
edge_prediction_df = edge_prediction_df[edge_testData.columns.astype(int)]
edge_prediction_df.head(2)

,1,10,100,101,102,106,107,108,109,11,...,90,91,92,93,94,95,96,97,98,99
0,1.401791,0.066712,3.966576,0.015035,0.248080,0.358919,9.370820,0.093440,-0.00544,0.105017,...,4.939750,0.278937,0.774766,0.244536,0.118174,1.122604,0.020343,1.156704,0.108951,-0.000374
1,0.634315,0.043807,1.794889,0.009872,0.114981,0.152434,3.541895,0.044782,-0.00268,0.050331,...,2.167781,0.139598,0.433944,0.136964,0.062533,0.628767,0.009429,0.491255,0.061023,-0.000184


In [35]:
r2_score(edge_testData.values, edge_prediction_df.values, multioutput='variance_weighted')

0.5071596126162997

In [ ]:
# 300 epoch MSE bptt 12 hidden=100
#0.4597512505107641

In [ ]:
# 150 epoch MSE bptt 12 hidden=100
# 0.47003788967204657

In [ ]:
# 150 epoch L1 bptt 12 hidden=100
# 0.49459994974800275

In [ ]:
# 300 epoch L1 bptt 12 hidden=100
# 0.48786324620679394

In [ ]:
# 150 epoch L1 bptt 6 hidden=100
# 0.5141071457595932

In [ ]:
# 150 epoch L1 bptt=6 hidden=50 
# 0.4721109077014878

In [ ]:
# 150 epoch L1 bptt=6 hidden=200 
# 0.4574538111782432

In [ ]:
# 300 epoch L1 bptt 6 hidden=100
# 0.5001768915084189

In [ ]:
# 150 epoch L1 bptt 6 hidden=100 layers 2
# 0.5157145438811139

In [ ]:
# 150 epoch L1 bptt 6 hidden=100 layers 3
# 0.5072893536554984